In [26]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from timeit import default_timer as timer
from scipy.special import softmax



In [13]:
df = pd.read_csv('ESS8_data.csv')

In [42]:
X = df[['SD1', 'PO1', 'UN1', 'AC1', 'SC1',
       'ST1', 'CO1', 'TR1', 'HD1', 'AC2', 'SC2', 'ST2',
       'CO2', 'PO2', 'BE2', 'TR2', 'HD2']].iloc[range(100),:]
X = X.to_numpy().T
N, M = X.T.shape
K = 3

In [43]:
np.random.seed(42)

p = np.random.permutation(range(N))[:K]
Z = X[:,p]
# Z = np.random.randint(1, 6, (M,K)) 

#Q = Z.T @ Z
#Qt = torch.tensor(Q,requires_grad=False).float()

R = X.T @ X
Rt = torch.tensor(R,requires_grad=False).float()

In [44]:
# applies both for eq. 7 and 8
def error(a_i,Qt,qt):
    return (0.5 * a_i.T @ Qt @ a_i) - (qt.T @ a_i)

In [45]:
def applyConstrains(M):
    return softmax(M, axis = 0)

In [46]:
start = timer()
torch.manual_seed(42)

RSS_values = list()
A = np.zeros((K,N))#.tolist()
B = np.zeros((N,K))#.tolist()

# LOOP UNTIL RSS IS LOW
for n in range(2): #N
    Q = Z.T @ Z
    Qt = torch.tensor(Q,requires_grad=False).float()

    # LOOP THROUGH ENTIRE A
    for i in range(N):
        q = Z.T @ X[:,i]
        qt = torch.tensor(q,requires_grad=False).float()

        a_i = torch.autograd.Variable(torch.rand(K, 1), requires_grad=True) # eller er det Kx1 ?

        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(a_i,Qt,qt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(10000): # 100000
            Delta = error(a_i,Qt,qt) 
            L = torch.norm(Delta, p=2)
            L.backward()
            a_i.data -= step_size * a_i.grad.data # step
            a_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss for a is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break
        
        A[:,i] = np.array(a_i.tolist()).flatten() 
        
        # print('Loss after: %s' % (torch.norm( error(a_i,Qt,qt) )))

    ### apply softmax here? ###
    
    A = applyConstrains(A)
    Z = X @ A.T @ np.linalg.inv(A@A.T)
    
    # LOOP THROUGH ENTIRE B
    for i in range(K): #K
        r = X.T @ Z[:,i]
        rt = torch.tensor(r,requires_grad=False).float()

        b_i = torch.autograd.Variable(torch.rand(N, 1), requires_grad=True) 

        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(b_i,Rt,rt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(10000): # 100000
            Delta = error(b_i,Rt,rt)
            L = torch.norm(Delta, p=2)
            L.backward()
            b_i.data -= step_size * b_i.grad.data # step
            b_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss for b is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break

        B[:,i] = np.array(b_i.tolist()).flatten() 
        #print('Loss after: %s' % (torch.norm( error(b_i,Rt,rt) )))    
    
    # apply softmax here
    B = applyConstrains(B)
    Z = X @ B
    
    Zt = torch.tensor(Z, requires_grad=False).float()
    At = torch.tensor(A, requires_grad=False).float()
    Xt = torch.tensor(X,requires_grad=False).float()
    print("RSS at n=%s" % n, torch.norm(Xt-Zt@At,p='fro'))
    RSS_values.append( torch.norm(Xt-Zt@At,p='fro'))
    
end = timer()

print("It took: {0} seconds to finish running".format(end - start))
print("The best RSS value was", min(RSS_values))

Loss before: tensor(12.1962, grad_fn=<NormBackward1>)
Loss for a is tensor(12.1962, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(81.5849, grad_fn=<NormBackward1>)
Loss for a is tensor(81.5849, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(47.7395, grad_fn=<NormBackward1>)
Loss for a is tensor(47.7395, grad_fn=<NormBackward1>) at iteration 0
It took 5406 iterations to achieve 1e-05 loss.
Loss before: tensor(10.3931, grad_fn=<NormBackward1>)
Loss for a is tensor(10.3931, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(39.5294, grad_fn=<NormBackward1>)
Loss for a is tensor(39.5294, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(9.2556, grad_fn=<NormBackward1>)
Loss for a is tensor(9.2556, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(90.6890, grad_fn=<NormBackward1>)
Loss for a is tensor(90.6890, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(54.3597, grad_fn=<NormBackward1>)
Loss for a is tensor(54.3597, gra

Loss before: tensor(61.7355, grad_fn=<NormBackward1>)
Loss for a is tensor(61.7355, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(48.9422, grad_fn=<NormBackward1>)
Loss for a is tensor(48.9422, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(28.6516, grad_fn=<NormBackward1>)
Loss for a is tensor(28.6516, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(53.7110, grad_fn=<NormBackward1>)
Loss for a is tensor(53.7110, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(50.1502, grad_fn=<NormBackward1>)
Loss for a is tensor(50.1502, grad_fn=<NormBackward1>) at iteration 0
It took 8018 iterations to achieve 1e-05 loss.
Loss before: tensor(79.6441, grad_fn=<NormBackward1>)
Loss for a is tensor(79.6441, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(0.9404, grad_fn=<NormBackward1>)
Loss for a is tensor(0.9404, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(50.8483, grad_fn=<NormBackward1>)
Loss for a is tensor(50.8483, gra

Loss before: tensor(19.6249, grad_fn=<NormBackward1>)
Loss for a is tensor(19.6249, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(35.9854, grad_fn=<NormBackward1>)
Loss for a is tensor(35.9854, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(47.9199, grad_fn=<NormBackward1>)
Loss for a is tensor(47.9199, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(42.7426, grad_fn=<NormBackward1>)
Loss for a is tensor(42.7426, grad_fn=<NormBackward1>) at iteration 0
It took 930 iterations to achieve 1e-05 loss.
Loss before: tensor(58.5071, grad_fn=<NormBackward1>)
Loss for a is tensor(58.5071, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(3.8124, grad_fn=<NormBackward1>)
Loss for a is tensor(3.8124, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(86.1826, grad_fn=<NormBackward1>)
Loss for a is tensor(86.1826, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(39.6422, grad_fn=<NormBackward1>)
Loss for a is tensor(39.6422, grad

It took 6454 iterations to achieve 1e-05 loss.
Loss before: tensor(61.0673, grad_fn=<NormBackward1>)
Loss for a is tensor(61.0673, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(99.7711, grad_fn=<NormBackward1>)
Loss for a is tensor(99.7711, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(10.9256, grad_fn=<NormBackward1>)
Loss for a is tensor(10.9256, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(29.9712, grad_fn=<NormBackward1>)
Loss for a is tensor(29.9712, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(47.2490, grad_fn=<NormBackward1>)
Loss for a is tensor(47.2490, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(38.2065, grad_fn=<NormBackward1>)
Loss for a is tensor(38.2065, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(36.0891, grad_fn=<NormBackward1>)
Loss for a is tensor(36.0891, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(45.0027, grad_fn=<NormBackward1>)
Loss for a is tensor(45.0027, g

### Time at different data sizes
### Here we run 1000 inner loops and 10 in the outer
#### For X = 3 x 17  
##### It took: 17.8 seconds


#### For X = 6 x 17  
##### It took: 28.08 seconds


#### For X = 12 x 17
#### It took: 48 seconds




In [180]:
Y = np.zeros((2,3))

In [182]:
Y[:,1] = np.array([1,1])

In [183]:
Y

array([[0., 1., 0.],
       [0., 1., 0.]])

In [41]:
A = torch.rand(2, 2, requires_grad=False)
b = torch.rand(2, 1,  requires_grad=False)

In [87]:
torch.inverse(A)

tensor([[ 2.1062, -1.4374],
        [-2.7958,  3.7027]])